# coreml_help - examples of use

This notebook shows examples of helper functions to facilitate working with
CoreML and ONNX and converting from one to the other.

I wrote these as a learning exercise. Feedback welcome.
Most of this is based on the work of others,  but there can be no question that any
bugs, errors, misstatements, and, especially, inept code constructs, are entirely mine.

>If you want **real** help with CoreML, I highly recommend **Matthijs Holleman's**
 *“Core ML Survival Guide.”*. Informative and well-written.
 Easy to read, as much as books on this subject can be.
  

**The next cell** is setup for what follows. You should execute it, but you can ignore its details.

In [1]:
# %load '../../lib/project_setup.py'
### Project Setup for Jupyter Notebook - customize this after first load
#
# Uncomment these after initial load (if you want them)
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%alias netron open -a /Applications/Netron.app

#Project Setup - Names - used as prefixs for several derived names below
project_name   = 'cml_help_example'  # 
model_name     = 'MobileNet'
batch_size     = 32
img_size       = 300

### Uncomment and change if you want to change the defaults
from ms_util import *
from pathlib import Path
local_root = Path('/Users/mcsieber/storage')  # On Paperspace would be just "/storage"
data_root  = Path('/Volumes/ArielD/storage')  # On Paperspace would be just "/storage"
user_lib   = local_root / 'lib'
notebooks  = local_root / 'notebooks'

### Run script to continue the setup ..
#run -i {user_lib}/complete_setup.py --show [ Proj Paths Env All None ] # defaults to 'Proj'
%run -i {user_lib}/finish_setup.py --show Proj

name 'defaults' is not defined

project_name    = cml_help_example
model_name      = MobileNet
proj_dir        = /Users/mcsieber/storage/notebooks/cml_help_example
proj_data_dir   = /Volumes/ArielD/storage/data/cml_help_example
models_dir      = /Volumes/ArielD/storage/data/cml_help_example/models
name 'fastai' is not defined
name 'torch' is not defined



Because we are not doing anything with Fastai in this notebook, it is ok that 'defaults', 'fastai' and 'torch' are not defined

#### Specify the location of the model file

In [2]:
mobilenet_dir  = data_root/'data/mlmodels/MobileNet'
mobilenet_path = mobilenet_dir/'MobileNet.mlmodel'
mobilenet_path

PosixPath('/Volumes/ArielD/storage/data/mlmodels/MobileNet/MobileNet.mlmodel')

In [3]:
import coremltools.models.model as cm
import coremltools.models.utils as cu
import coreml_help
from coreml_help import *

#### The browser object will get and save several values that are useful to keep around: 
- the protobuf spec, 
- the neural network object, 
- the array of neural network layers and its lenght
- a dictionary mapping layer names to layer indexes
- the shape inference object for the model

In [4]:
mnb = CoremlBrowser(mobilenet_path)

Using shape info from compilation output


It also overrides the default repr so that something more useful than "object" is shown:

In [5]:
mnb

mlmodel_path      = /Volumes/ArielD/storage/data/mlmodels/MobileNet/MobileNet.mlmodel
layer_count       = 111
layer_shapes_count = 112
layer_count       = input {
  name: "data"
  shortDescription: "Input image to be classified"
  type {
    imageType {
      width: 224
      height: 224
      colorSpace: BGR
    }
  }
}
output {
  name: "prob"
  shortDescription: "Probability of each category"
  type {
    dictionaryType {
      stringKeyType {
      }
    }
  }
}
output {
  name: "classLabel"
  shortDescription: "Most likely image category"
  type {
    stringType {
    }
  }
}
predictedFeatureName: "classLabel"
predictedProbabilitiesName: "prob"
metadata {
  shortDescription: "The network from the paper \'MobileNets: Efficient Convolutional Neural Networks for Mobile Vision Applications\', trained on the ImageNet dataset."
  author: "Original paper: Andrew G. Howard, Menglong Zhu, Bo Chen, Dmitry Kalenichenko, Weijun Wang, Tobias Weyand, Marco Andreetto, Hartwig Adam. Caffe implemen

### Model Inspection
* `show_nn` will show a slice of the layers.  
*  e.g. `show_nn(mnb,8,2)` starts at layer 8 and shows 2 layers. `mnb.show_nn(8,2)` is  equivalent
* `show_head` shows the first 3 layers.  It is a convenience routine for `mnb.show_nn(0,3)`
* `show_tail` shows the last 3 layers.

*(you will probably need to resize your browser to full width to show the lines well)*

In [6]:
show_head(mnb)

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
  0 conv1                    [data]                          conv     32x3       sz:3x3  str:2x2  dil:1x1  wc:864
                             [conv1_1conv1/bn]                c h w:  32 112 112

  1 conv1/bn                 [conv1_1conv1/bn]               bnorm    32         ep:1.000e-05  wc:64
                             [conv1_2conv1/scale]             c h w:  32 112 112

  2 conv1/scale              [conv1_2conv1/scale]                   - 
                             [conv1_3relu1]                   c h w:  32 112 112



In [7]:
show_nn(mnb,8,2)  # or mnb.show_nn(8,2) will show the same thing

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
  8 conv2_1/sep              [conv2_1/dw]                    conv     64x32      sz:1x1  str:1x1  dil:1x1  wc:2048
                             [conv2_1/sep_9conv2_1/sep/bn]    c h w:  64 112 112

  9 conv2_1/sep/bn           [conv2_1/sep_9conv2_1/sep/bn]   bnorm    64         ep:1.000e-05  wc:128
                             [conv2_1/sep_10conv2_1/sep/sca   c h w:  64 112 112



In [8]:
show_tail(mnb)

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
108 pool6                    [conv6/sep]                     pool                sz:0x0  str:1x1
                             [pool6]                          c h w:  1024 1  1 

109 fc7                      [pool6]                         conv     1000x1024  sz:1x1  str:1x1  dil:1x1  wc:1024000
                             [fc7]                            c h w:  1000 1  1 

110 prob                     [fc7]                                  - 
                             [prob]                           c h w:  1000 1  1 



#### Netron

The best way to view the whole model is to use Netron.  If you have it installed and are using MacOS, `%netron` is a "magic command" alias (see setup above) that will launch it for you

In [9]:
%netron {mobilenet_path}

### Layer Access
The method `get_layer` will retrieve a layer by number (index) or name.

In [ ]:
# Get layer by name
layer_pool6 = mnb.get_layer('pool6')
# Get layer by name
layer_10 = mnb.get_layer(10)

### Model Surgery

Let's connect layer **pool6** to layer **prob** jumping over layer **fc7**

* `connect_layers` returns a named tuple recording what it just did. 
(Just in case you want to keep track, or later decide to undo it ...)
* When we connect layers, the only data that is changed is the "input" for the "to" layer.

The next three cells show "before", the connect operation, and "after"

In [10]:
show_tail(mnb)

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
108 pool6                    [conv6/sep]                     pool                sz:0x0  str:1x1
                             [pool6]                          c h w:  1024 1  1 

109 fc7                      [pool6]                         conv     1000x1024  sz:1x1  str:1x1  dil:1x1  wc:1024000
                             [fc7]                            c h w:  1000 1  1 

110 prob                     [fc7]                                  - 
                             [prob]                           c h w:  1000 1  1 



In [11]:
mnb.connect_layers(from_='pool6', to_='prob')

layer_audit(changed_layer='prob', input_before=['fc7'], input_after=['pool6'], error=None)

In [12]:
show_tail(mnb)

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
108 pool6                    [conv6/sep]                     pool                sz:0x0  str:1x1
                             [pool6]                          c h w:  1024 1  1 

109 fc7                      [pool6]                         conv     1000x1024  sz:1x1  str:1x1  dil:1x1  wc:1024000
                             [fc7]                            c h w:  1000 1  1 

110 prob                     [pool6]                                - 
                             [prob]                           c h w:  1000 1  1 



In the layers above, you can see that layer **prob** now gets input from the output of layer **pool6**.  Layer **fc7** has been isolated - tho it is still receiving input from **pool6**, there is now no layer that takes its output.

So lets delete **fc7** . (You can delete multiple layers at once, if you wish.) 
Again, the method returns a record of the change.  

In [13]:
mnb.delete_layers(['fc7'])

[{'deleted_layer': 'fc7', 'input': ['pool6'], 'output': ['fc7']}]

Layer **fc7** is now gone ...

In [14]:
show_tail(mnb)

Lay Name                     In/Out                          Type,Chan(s)/Shape   Size,Stride,Dilation,#Wts
107 relu6/sep                [conv6/sep_107relu6/sep]        ReLU     
                             [conv6/sep]                      c h w:  1024 7  7 

108 pool6                    [conv6/sep]                     pool                sz:0x0  str:1x1
                             [pool6]                          c h w:  1024 1  1 

109 prob                     [pool6]                                - 
                             [prob]                           c h w:  1000 1  1 



To recompile (and save) the model after surgery ...

In [ ]:
mnet_mlmodel = mnb.compile_spec()  # equivalent to mnet_mlmodel = MLModel(mnb.spec)

###  Surgery Errors

* For `connect_layers`, if the either of the names is invalid, the operation is aborted
* For `delete_layers`  invalid names are skipped and silently ignored

In [15]:
mnb.connect_layers(from_='pool28', to_='xyzzy')

layer_audit(changed_layer='NONE', input_before=None, input_after=None, error="Layer(s) [['pool28', 'xyzzy']] not found")

In [16]:
mnb.delete_layers(['pool6','relu6/sep','pool28','xyzzy'])



[{'deleted_layer': 'pool6', 'input': ['conv6/sep'], 'output': ['pool6']},
 {'deleted_layer': 'relu6/sep', 'input': ['conv6/sep_107relu6/sep'], 'output': ['conv6/sep']}]